In [2]:
from sage.crypto.block_cipher.miniaes import MiniAES

# Voy a trabajar con bin(). Por tanto, todas las funciones van a esperar eso

K = FiniteField(16, 'xi')
MS = MatrixSpace(K, 2, 2)
miniaes = MiniAES()
bin = BinaryStrings()

dni = 77432071

# Apartado 1

In [161]:
def xor(a, b):
    f = miniaes.binary_to_GF(a)[0]
    g = miniaes.binary_to_GF(b)[0]
    
    return miniaes.GF_to_binary(f + g)

def gamma(a):
    if   a == bin("0000"):
        return bin("0011")
    elif a == bin("0001"):
        return bin("1000")
    elif a == bin("0010"):
        return bin("1111")
    elif a == bin("0011"):
        return bin("0111")
    
    elif a == bin("0100"):
        return bin("0001")
    elif a == bin("0101"):
        return bin("0010")
    elif a == bin("0110"):
        return bin("1011")
    elif a == bin("0111"):
        return bin("0000")
    
    elif a == bin("1000"):
        return bin("1100")
    elif a == bin("1001"):
        return bin("1110")
    elif a == bin("1010"):
        return bin("1010")
    elif a == bin("1011"):
        return bin("0110")
    
    elif a == bin("1100"):
        return bin("1001")
    elif a == bin("1101"):
        return bin("1101")
    elif a == bin("1110"):
        return bin("0101")
    elif a == bin("1111"):
        return bin("0100")
    
def sub_gamma(A):
    return matriz(
        gamma(A[0][0]), gamma(A[1][0]), gamma(A[0][1]), gamma(A[1][1])
    )

def shift_row(A):
    return matriz(
        A[0][0], A[1][1], A[0][1], A[1][0]
    )

def theta(A_GF):
    """
    Recibe una matriz en GF, y devuelve 
    la multiplicación correspondiente en GF
    """
    B = matriz_binary_to_GF(
        bin("0011"), bin("0010"), bin("0010"), bin("0011")
    )
    
    a0 = B[0][0] * A_GF[0][0] + B[0][1] * A_GF[1][0]
    a1 = B[1][0] * A_GF[0][0] + B[1][1] * A_GF[1][0]
    a2 = B[0][0] * A_GF[0][1] + B[0][1] * A_GF[1][1]
    a3 = B[1][0] * A_GF[0][1] + B[1][1] * A_GF[1][1]
    
    return matriz(a0, a1, a2, a3)

def sigma(A, K):
    """
    A está en binario, K es la clave (array de 4 posiciones)
    """
    a0 = xor(A[0][0], K[0])
    a1 = xor(A[1][0], K[1])
    a2 = xor(A[0][1], K[2])
    a3 = xor(A[1][1], K[3])
    
    return matriz(
        a0, a1, a2, a3
    )

In [182]:
def matriz(a0, a1, a2, a3):
    return [
        [a0, a2],
        [a1, a3]
    ]

def matriz_binary_to_GF(a0, a1, a2, a3):
    """
    Recibe números en binario, y construye una matriz en GF
    """
    return matriz(
        miniaes.binary_to_GF(a0)[0], 
        miniaes.binary_to_GF(a1)[0], 
        miniaes.binary_to_GF(a2)[0], 
        miniaes.binary_to_GF(a3)[0]
    )

def matriz_to_GF(A):
    return matriz(
        miniaes.binary_to_GF(A[0][0])[0], 
        miniaes.binary_to_GF(A[1][0])[0], 
        miniaes.binary_to_GF(A[0][1])[0], 
        miniaes.binary_to_GF(A[1][1])[0]
    )

def matriz_GF_to_binary(A):
    return matriz(
        bin(miniaes.GF_to_binary(A[0][0])), 
        bin(miniaes.GF_to_binary(A[1][0])), 
        bin(miniaes.GF_to_binary(A[0][1])), 
        bin(miniaes.GF_to_binary(A[1][1]))
    )

In [159]:
def calcular_claves(clave, modulo = 65536):
    clave_en_binario = clave.mod(65536).binary()

    w0 = bin(clave_en_binario[0:4])
    w1 = bin(clave_en_binario[4:8])
    w2 = bin(clave_en_binario[8:12])
    w3 = bin(clave_en_binario[12:16])

    w4 = xor(xor(w0, gamma(w3)), bin("0001"))
    w5 = xor(w1, w4)
    w6 = xor(w2, w5)
    w7 = xor(w3, w6)

    w8  = xor(xor(w4, gamma(w7)), bin("0010"))
    w9  = xor(w5, w8)
    w10 = xor(w6, w9)
    w11 = xor(w7, w10)


    K0 = [w0, w1, w2, w3]
    K1 = [w4, w5, w6, w7]
    K2 = [w8, w9, w10, w11]

    return K0, K1, K2

In [174]:
def pasada_E_k(A, K0, K1, K2):
    q1 = sigma(A, K0)
    print("sigma_K0()  = " + str(q1))
    
    q2 = sub_gamma(q1)
    print("gamma()     = " + str(q2))
    
    q3 = shift_row(q2)
    print("pi()        = " + str(q3))
    
    q4 = matriz_to_GF(q3)
    print("pi() en GF  = " + str(q4))
    
    q5 = theta(q4)
    print("theta()     = " + str(q5))
    
    q6 = matriz_GF_to_binary(q5)
    print("theta() en binario = " + str(q6))
    
    q7 = sigma(q6, K1)
    print("sigma_K1()  = " + str(q7))
    
    q8 = sub_gamma(q7)
    print("gamma()     = " + str(q8))
    
    q9 = shift_row(q8)
    print("pi()        = " + str(q9))
    
    q10 = sigma(q9, K2)
    print("gamma_K2()  = " + str(q10))
    
    return q10

In [175]:
K0, K1, K2 = calcular_claves(dni)
K0, K1, K2

([1000, 0101, 0000, 0111], [1001, 1100, 1100, 1011], [1101, 0001, 1101, 0110])

In [176]:
# m = 0000 0001 0010 0011 0100 0101 0110 0111
m = bin("00000001001000110100010101100111")

In [177]:
# p0 = m1 xor c0
p0 = matriz(
    bin("0000"), bin("0001"), bin("0010"), bin("0010")
)

pasada_E_k(p0, K0, K1, K2)

sigma_K0()  = [[1000, 0010], [0100, 0101]]
gamma()     = [[1100, 1111], [0001, 0010]]
pi()        = [[1100, 1111], [0010, 0001]]
pi() en GF  = [[x^3 + x^2, x^3 + x^2 + x + 1], [x, 1]]
theta()     = [[x + 1, 0], [x^3 + x^2 + 1, x^3 + x^2 + x]]
theta() en binario = [[0011, 0000], [1101, 1110]]
sigma_K1()  = [[1010, 1100], [0001, 0101]]
gamma()     = [[1010, 1001], [1000, 0010]]
pi()        = [[1010, 1001], [0010, 1000]]
gamma_K2()  = [[0111, 0100], [0011, 1110]]


[[0111, 0100], [0011, 1110]]

Segunda ronda

In [178]:
# q0 = m2 xor c1
q0 = matriz(
    bin("0011"), bin("0110"), bin("0010"), bin("1001") 
)
q0

[[0011, 0010], [0110, 1001]]

In [179]:
pasada_E_k(q0, K0, K1, K2)

sigma_K0()  = [[1011, 0010], [0011, 1110]]
gamma()     = [[0110, 1111], [0111, 0101]]
pi()        = [[0110, 1111], [0101, 0111]]
pi() en GF  = [[x^2 + x, x^3 + x^2 + x + 1], [x^2 + 1, x^2 + x + 1]]
theta()     = [[0, x^3 + x^2], [x + 1, x^2]]
theta() en binario = [[0000, 1100], [0011, 0100]]
sigma_K1()  = [[1001, 0000], [1111, 1111]]
gamma()     = [[1110, 0011], [0100, 0100]]
pi()        = [[1110, 0011], [0100, 0100]]
gamma_K2()  = [[0011, 1110], [0101, 0010]]


[[0011, 1110], [0101, 0010]]

# Apartado 2

In [180]:
IV = matriz(
    bin("0000"), bin("0000"), bin("0000"), bin("0001")
)

In [181]:
pasada_E_k(IV, K0, K1, K2)

sigma_K0()  = [[1000, 0000], [0101, 0110]]
gamma()     = [[1100, 0011], [0010, 1011]]
pi()        = [[1100, 0011], [1011, 0010]]
pi() en GF  = [[x^3 + x^2, x + 1], [x^3 + x + 1, x]]
theta()     = [[x, 1], [x^2 + 1, 0]]
theta() en binario = [[0010, 0001], [0101, 0000]]
sigma_K1()  = [[1011, 1101], [1001, 1011]]
gamma()     = [[0110, 1101], [1110, 0110]]
pi()        = [[0110, 1101], [0110, 1110]]
gamma_K2()  = [[1011, 0000], [0111, 1000]]


[[1011, 0000], [0111, 1000]]